In [33]:
# yield from <subgenerator>
# 作用是把不同的生成器结合在一起使用
# 引入yield from 是为了支持实现了__next__、send、close、和throw方法的生成器
# 需要注意这几个概念：客户端 - 委托生成器 - 子生成器
# yield form在客户端和子生成器之间提供了一个管道可以直接发送和产出值
# 
# 如计算平均值的例子

data = {
    'item1':[6,14,8,24,26,18,20,34,71],
    'item2':[27,6,81,25,19,8,22,15,23],
    'item3':[19,32,8,4,39,53,17,13,14],
}

# 子生成器
# 可以看做一个协程，可以挂起suspend与恢复resume
def subGenerator():
    count   = 0
    total   = 0
    average = 0
    while True:
        term = yield # 接收客户端send()的值
        if term == None:
            break
        total += term
        count += 1
        average = total/count
    return (count,average)


# 委派生成器
def tubeGenerator(results,key):
    while True: # 如果不加循环，就只能执行一次next或者send，没法给子生成器发送数据了，虽然可能会造成浪费 每次都多生成一个实例
        results[key] = yield from subGenerator() # 停在yield from，等待subGenerator()的返回值，赋给results

# 客户端代码
def client(data):
    results = {}
    for key,values in data.items():
        Tube = tubeGenerator(results,key) # 创建委托生成器实例Tube，上一次创建的Tube则被垃圾回收
        next(Tube) # 启动委托生成器，并停在yield from
        for value in values:
            Tube.send(value) # 直接发送到子生成器的term，委托生产起感知不到这个值
        Tube.send(None) # 结束子生成器，否则Tube会一直被卡住，拿不到返回值
    return results

results = client(data)
print(results)


{'item1': (9, 24.555555555555557), 'item2': (9, 25.11111111111111), 'item3': (9, 22.11111111111111)}


In [ ]:
# test2
# 如果yield from后跟的不是子生成器，而是一个可迭代对象，那么作用就基本等于yield
# 如下例
def func2():
    # while True:
        yield from data;

for i in func2():
    print(i)
